In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from geopy.distance import geodesic
from bokeh.plotting import figure, output_notebook, show
from bokeh.models import ColumnDataSource
import time

In [ ]:
df = pd.read_csv("dataset.csv")
df = df.sort_values(by=["LastUpdatedDate", "LastUpdatedTime", "SystemCodeNumber"]).reset_index(drop=True)
df["Price"] = 10.0  # Initial base price

In [ ]:
def demand_based_price(base_price, occupancy, capacity, queue, traffic, special_day, vehicle_type,
                       weights, lambda_=0.5):
    demand = (
        weights['occupancy'] * (occupancy / capacity) +
        weights['queue'] * queue -
        weights['traffic'] * traffic +
        weights['special_day'] * special_day +
        weights['vehicle_type'] * vehicle_type_weights.get(vehicle_type, 1.0)
    )
    norm_demand = max(0, min(demand, 1))
    price = base_price * (1 + lambda_ * norm_demand)
    return max(base_price * 0.5, min(price, base_price * 2))

In [ ]:
def baseline_price(prev_price, occupancy, capacity, alpha=2.0):
    return prev_price + alpha * (occupancy / capacity)

In [ ]:
def get_nearby_lots(current_lot, all_lots, max_distance_km=0.5):
    current_coords = (all_lots[current_lot]["Latitude"], all_lots[current_lot]["Longitude"])
    nearby = []
    for other_lot, coords in all_lots.items():
        if other_lot != current_lot:
            dist = geodesic(current_coords, (coords["Latitude"], coords["Longitude"])).km
            if dist <= max_distance_km:
                nearby.append(other_lot)
    return nearby


In [ ]:
lot_locations = df.groupby("SystemCodeNumber")[["Latitude", "Longitude"]].first().to_dict("index")
weights = {'occupancy': 0.5, 'queue': 0.2, 'traffic': 0.1, 'special_day': 0.3, 'vehicle_type': 0.4}
prev_prices = {lot: 10.0 for lot in df["SystemCodeNumber"].unique()}

updated_prices = []

for i in range(len(df)):
    row = df.loc[i]
    lot_id = row["SystemCodeNumber"]
    occupancy, capacity = row["Occupancy"], row["Capacity"]
    queue, traffic = row["QueueLength"], row["TrafficConditionNearby"] # Get the traffic condition string
    special_day, vehicle_type = row["IsSpecialDay"], row["VehicleType"]
    base_price = prev_prices[lot_id]

    # Convert traffic condition to a numerical value for the demand calculation
    traffic_value = 0
    if traffic == 'low':
        traffic_value = 0.1
    elif traffic == 'average':
        traffic_value = 0.5
    elif traffic == 'high':
        traffic_value = 1.0

    # Demand-based pricing
    price = demand_based_price(base_price, occupancy, capacity, queue, traffic_value, special_day,
                               vehicle_type, weights)

    # Competitive adjustment
    # Ensure the lot_locations dictionary has all unique lot IDs from the DataFrame
    # If not, you might need to update it or handle missing keys
    nearby_lots = get_nearby_lots(lot_id, lot_locations)
    cheaper_competitor = any(prev_prices.get(n, 10.0) < price for n in nearby_lots)
    if occupancy >= capacity and cheaper_competitor:
        price *= 0.95
    elif all(prev_prices.get(n, 10.0) > price for n in nearby_lots):
        price *= 1.05

    price = max(5, min(price, 20))
    updated_prices.append(price)
    prev_prices[lot_id] = price

df["UpdatedPrice"] = updated_prices

In [ ]:
lot_id = df["SystemCodeNumber"].unique()[0]
lot_df = df[df["SystemCodeNumber"] == lot_id].reset_index(drop=True)

source = ColumnDataSource(data=dict(x=[], y=[]))
p = figure(title=f"Real-Time Pricing – Lot {lot_id}", x_axis_label="Time Step", y_axis_label="Price")
p.line('x', 'y', source=source, line_width=2)
output_notebook() # Add this line to display the plot in the notebook
show(p)

for i in range(len(lot_df)):
    # Assuming 'UpdatedPrice' column exists from previous calculations
    current_price = lot_df.loc[i, "UpdatedPrice"]
    new_data = {'x': [i], 'y': [current_price]}
    source.stream(new_data, rollover=100)
    print(f"⏱️ Time Step: {i}, Price: {round(current_price, 2)}")
    time.sleep(0.2)


In [ ]:
occupancy = lot_df.loc[i, "Occupancy"]
queue = lot_df.loc[i, "QueueLength"]
traffic = lot_df.loc[i, "TrafficConditionNearby"]

print(f"⏱️ Time Step: {i}, Price: {round(current_price, 2)}, Occupancy: {occupancy}, Queue: {queue}, Traffic: {traffic}")